In [0]:

from google.colab import drive 
 
drive.mount('/content/gdrive')

!unzip "/content/gdrive/My Drive/miscellaneous/data1-2.h5.zip"

!ls

import numpy as np 
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt


def build_baseline(width, height, depth, classes):

  # initializing sequential model
  model = tf.keras.Sequential()

  # input shape is always required in the first layer
  inputShape = (height, width, depth)
  
  # define the first and only CONV => POOL layer
  model.add(tf.keras.layers.Conv2D (32, (3, 3), padding="same",input_shape=inputShape, activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

  # softmax classifier to identify the 17 classes of Flowers
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  return model

def build_arch_2(width, height, depth, classes):

  # initializing sequential model
  model = tf.keras.Sequential()
  
  # input shape is always required in the first layer
  inputShape = (height, width, depth)
  
  # define the first CONV => POOL layer
  model.add(tf.keras.layers.Conv2D (32, (3, 3), padding="same",input_shape=inputShape, activation='relu'))  
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  
  # define the second CONV => POOL layer
  model.add(tf.keras.layers.Conv2D(32, (3, 3), padding="same",activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))  
  
  # define the third CONV => POOL layer
  model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same",activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))  

  # define the only FC => RELU layer
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(512,activation='relu'))
  model.add(tf.keras.layers.Dropout(rate=0.2))    
  
  # softmax classifier to identify the 17 classes of Flowers
  model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  return model

def build_arch_3(width, height, depth, classes):

  # initializing sequential model
  model = tf.keras.Sequential()

  # input shape is always required in the first layer
  inputShape = (height, width, depth)
  
  # define the first CONV => CONV => POOL layer
  model.add(tf.keras.layers.Conv2D(32, (3, 3), padding="same",input_shape=inputShape, activation='relu'))
  model.add(tf.keras.layers.Conv2D(32, (3, 3), padding="same",activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  
  # define the second CONV => CONV => POOL layer
  model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same",activation='relu'))
  model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same",activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  
  # define the only FC => RELU layer
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(512,activation='relu'))
  model.add(tf.keras.layers.Dropout(rate=0.2)) 
  
  # softmax classifier to identify the 17 classes of Flowers
  model.add(tf.keras.layers.Dense(classes, activation='softmax'))
 

  return model


def loadDataH5():
  
  with h5py.File('data1.h5','r') as hf:
    trainX = np.array(hf.get('trainX')) 
    trainY = np.array(hf.get('trainY')) 
    valX = np.array(hf.get('valX')) 
    valY = np.array(hf.get('valY')) 
    print (trainX.shape,trainY.shape) 
    print (valX.shape,valY.shape)
    
    return trainX, trainY, valX, valY
  
def cnn_augmentation():
  
  # num of epochs for the CNN augmentation. It is higher than the normal as dataset with augmentation took longer to diverge
  NUM_EPOCHS = 100

  # load the training and testing data
  trainX, trainY, testX, testY = loadDataH5()

  # initialize the optimizer and model
  print("Compiling model...")
  opt = tf.keras.optimizers.SGD(lr=0.01)

  model = build_arch_2(width=128, height=128, depth=3, classes=17)

  print (model.summary())

  model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

  # data augmentation for training data
  trainDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=15,
      #width_shift_range=0.1,
      #height_shift_range=0.1,
      horizontal_flip=True,
      )

  train_generator = trainDataGenerator.flow(
    trainX,
    trainY,
    batch_size=32)

  # data augmentation for training data for testing data, which is empty as data for validation is already scaled and no changes are needed.
  valDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator()

  validation_generator = valDataGenerator.flow(
    testX,
    testY,
    batch_size=32)

  H = model.fit_generator(train_generator,
         steps_per_epoch= 1020 // 32,
          epochs=NUM_EPOCHS,
          validation_data=validation_generator,
          validation_steps=340 // 32                       
         )

def cnn():
  
  NUM_EPOCHS = 60

  # load the training and testing data
  trainX, trainY, testX, testY = loadDataH5()

  # initialize the optimizer and model
  print("Compiling model...")
  opt = tf.keras.optimizers.SGD(lr=0.01)

  #model = build_baseline(width=128, height=128, depth=3, classes=17)
  model = build_arch_2(width=128, height=128, depth=3, classes=17)
  #model = build_arch_3(width=128, height=128, depth=3, classes=17)

  print (model.summary())

  model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
  

  # train the network
  print("Training network...")
  H = model.fit(trainX, trainY, validation_data=(testX, testY),batch_size=32, epochs=NUM_EPOCHS)


  
# executing CNN  
cnn_augmentation()

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 60), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 60), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 60), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 60), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Archive:  /content/gdrive/My Drive/miscellaneous/data1-2.h5.zip
replace data1.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
data1.h5  gdrive  sample_data
(1020, 128, 128, 3) (1020,)
(340, 128, 128, 3) (340,)
Compiling model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 32, 32, 32)        0         
__________________________________________________

In [15]:
# Ensemble - The Architecture 2 with data augmentation will be used
# Only one model is being used. This will cause a lack of diversity and results will be very similar

def model_holder(trainX, trainY):
  
  
  opt = tf.keras.optimizers.SGD(lr=0.01)
  model = build_arch_2(width=128, height=128, depth=3, classes=17)
  
  model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
  
  # data augmentation for training data
  trainDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=15,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      )

  train_generator = trainDataGenerator.flow(
    trainX,
    trainY,
    batch_size=32)  
  
  H = model.fit(train_generator, batch_size=32, epochs=NUM_EPOCHS)  
  
  return model

# comparing predicted with true y to get accuracy
def accuracy_score(predicted_y, true_y):
  
  predictions_correct = np.equal(predicted_y, true_y)
  
  # sum of correct / len of items
  accuracy = np.sum(predictions_correct.astype(int)) / len(predictions_correct)
  
  
  return accuracy

# main function to gather accuracy for individual members and the overall ensemble
def ensemble_accuracy(members_probs):
  
  base_members_accuracy = []
  
  for i, member_probs in enumerate(members_probs):
    
    base_pred_y = np.argmax(member_probs, axis=1)
    accuracy = accuracy_score(base_pred_y, testY)
    
    print("Base learner", i + 1, "accuracy: ", accuracy) 

  members_probs = np.array(members_probs)
  
  # argmax against ensemble
  ensemble_probs = np.sum(members_probs, axis=0)
  ensemble_pred_y = np.argmax(ensemble_probs, axis=1)  
  
  # calculate ensemble accuracy
  ensemble_accuracy = accuracy_score(ensemble_pred_y, testY)
  
  print("Final accuracy:", ensemble_accuracy)
  
# load the training and testing data
trainX, trainY, testX, testY = loadDataH5()  

NUM_EPOCHS = 60  
base_learners = 5

# create members model placeholder
members = []

for i in range(base_learners):
  member = model_holder(trainX, trainY)
  
  # appending each model to a list of models
  members.append(member)

# Make predictions for each member
members_probs = []

for model in members:
  
  members_probs.append(model.predict(testX)) 
  
  
# call main function to print ensemble accuracy
ensemble_accuracy(members_probs)    

(1020, 128, 128, 3) (1020,)
(340, 128, 128, 3) (340,)
Epoch 1/60
32/32 [==============================] - 2s 71ms/step - loss: 2.8332 - acc: 0.0706
Epoch 2/60
32/32 [==============================] - 3s 101ms/step - loss: 2.7995 - acc: 0.1059
Epoch 3/60
32/32 [==============================] - 3s 102ms/step - loss: 2.7213 - acc: 0.1373
Epoch 4/60
32/32 [==============================] - 3s 100ms/step - loss: 2.5245 - acc: 0.1951
Epoch 5/60
32/32 [==============================] - 3s 101ms/step - loss: 2.3237 - acc: 0.2304
Epoch 6/60
32/32 [==============================] - 3s 102ms/step - loss: 2.2058 - acc: 0.2667
Epoch 7/60
32/32 [==============================] - 3s 100ms/step - loss: 2.1180 - acc: 0.2814
Epoch 8/60
32/32 [==============================] - 3s 100ms/step - loss: 1.9975 - acc: 0.3088
Epoch 9/60
32/32 [==============================] - 3s 102ms/step - loss: 1.9367 - acc: 0.3431
Epoch 10/60
32/32 [==============================] - 3s 101ms/step - loss: 1.8895 - acc: 0.3